# This script:
**Detects balls with changable trackbars**

In [ ]:
'''
This program detects balls (either white or orange) in an image using color filtering and Hough Circle detection.
The user can switch between detecting white or orange balls by pressing the 'b' key. The program uses 
trackbars to adjust the HSV color range for ball detection and Hough Circle parameters for accurate circle 
detection. The image is processed within a defined bounding box, and the program displays the detected balls, 
their positions, and the number of balls detected in a live window.
The user can exit the program by pressing the 'q' key.
'''



import cv2
import numpy as np

# Boolean to switch between white and orange ball detection
white = False  

# Load the image
image = cv2.imread("Udklip.PNG")
height, width = image.shape[:2]

# Resize while keeping aspect ratio
new_width = 800
new_height = int((new_width / width) * height)
image = cv2.resize(image, (new_width, new_height))

# Define bounding box coordinates (Modify these as needed)
x1, y1 = 0+20, 0+20  # Top-left corner
x2, y2 = new_width-20, new_height-20  # Bottom-right corner

# Create windows
cv2.namedWindow("Trackbars", cv2.WINDOW_NORMAL)
cv2.namedWindow("Ball Detection", cv2.WINDOW_NORMAL)
cv2.namedWindow("Binary Mask", cv2.WINDOW_NORMAL)

# Move windows
cv2.moveWindow("Trackbars", 0, 0)
cv2.moveWindow("Ball Detection", 605, 0)
cv2.moveWindow("Binary Mask", 1100, 0)

# Function to do nothing (for trackbars)
def nothing(x):
    pass

# Function to create trackbars based on ball color
def create_trackbars(white):
    if white:
        cv2.createTrackbar("Hue min", "Trackbars", 0, 180, nothing)
        cv2.createTrackbar("Hue max", "Trackbars", 180, 180, nothing)
        cv2.createTrackbar("Sat min", "Trackbars", 0, 255, nothing)
        cv2.createTrackbar("Sat max", "Trackbars", 20, 255, nothing)
        cv2.createTrackbar("Bright min", "Trackbars", 180, 255, nothing)
        cv2.createTrackbar("Bright max", "Trackbars", 255, 255, nothing)
    else:
        cv2.createTrackbar("Hue min", "Trackbars", 15, 180, nothing)
        cv2.createTrackbar("Hue max", "Trackbars", 25, 180, nothing)
        cv2.createTrackbar("Sat min", "Trackbars", 190, 255, nothing)
        cv2.createTrackbar("Sat max", "Trackbars", 255, 255, nothing)
        cv2.createTrackbar("Bright min", "Trackbars", 190, 255, nothing)
        cv2.createTrackbar("Bright max", "Trackbars", 255, 255, nothing)

# Initialize trackbars for white balls
create_trackbars(white)

# Trackbars for HoughCircles parameters
cv2.createTrackbar("p1 Canny Edge", "Trackbars", 10, 255, nothing)
cv2.createTrackbar("p2 Sensitivity", "Trackbars", 10, 100, nothing)
cv2.createTrackbar("Ball min", "Trackbars", 4, 100, nothing)
cv2.createTrackbar("Ball max", "Trackbars", 10, 200, nothing)

while True:
    # Get trackbar values
    lower_h = cv2.getTrackbarPos("Hue min", "Trackbars")
    upper_h = cv2.getTrackbarPos("Hue max", "Trackbars")
    lower_s = cv2.getTrackbarPos("Sat min", "Trackbars")
    upper_s = cv2.getTrackbarPos("Sat max", "Trackbars")
    lower_v = cv2.getTrackbarPos("Bright min", "Trackbars")
    upper_v = cv2.getTrackbarPos("Bright max", "Trackbars")

    param1 = max(1, cv2.getTrackbarPos("p1 Canny Edge", "Trackbars"))
    param2 = max(1, cv2.getTrackbarPos("p2 Sensitivity", "Trackbars"))
    min_radius = cv2.getTrackbarPos("Ball min", "Trackbars")
    max_radius = cv2.getTrackbarPos("Ball max", "Trackbars")

    # Convert image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply bounding box (crop image to selected area)
    roi = hsv[y1:y2, x1:x2]  # Region of interest

    # Apply color filter
    lower_bound = np.array([lower_h, lower_s, lower_v])
    upper_bound = np.array([upper_h, upper_s, upper_v])
    mask = cv2.inRange(roi, lower_bound, upper_bound)
    blurred = cv2.GaussianBlur(mask, (9, 9), 2)

    # Detect circles
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=param1, param2=param2, 
                               minRadius=min_radius, maxRadius=max_radius)

    # Draw detected balls on the original image
    output = image.copy()
    ball_count = 0

    if circles is not None:
        circles = np.uint16(np.around(circles))
        ball_count = len(circles[0, :])

        # Prepare position info
        positions_text = [f"({x + x1}, {y + y1})" for x, y, r in circles[0, :]]  # Adjust to global coordinates
        positions_text = ", ".join(positions_text)

        for (x, y, r) in circles[0, :]:
            global_x, global_y = x + x1, y + y1  # Convert to original image coordinates
            cv2.circle(output, (global_x, global_y), r, (0, 255, 0), 2)
            cv2.putText(output, "Ball", (global_x, global_y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Draw bounding box
    cv2.rectangle(output, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red box around detection area
    cv2.putText(output, "Detection Zone", (x1, y1 - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display ball count and positions on screen
    cv2.putText(output, f"Balls detected: {ball_count}", (10, new_height - 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(output, "Positions: " + positions_text, (10, new_height - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Show results
    cv2.imshow("Ball Detection", output)
    cv2.imshow("Binary Mask", mask)

    # Switch between detecting white and orange balls when 'b' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('b'):
        white = not white
        create_trackbars(white)

    # Press 'q' to exit
    if key == ord('q'):
        break

cv2.destroyAllWindows()


# This script:
**Detects walls, corners and creates coordinate system**

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Load the image
image = cv2.imread("Udklip2.jpeg")
height, width = image.shape[:2]

# Resize while keeping aspect ratio
new_width = 800
new_height = int((new_width / width) * height)
image = cv2.resize(image, (new_width, new_height))

# Convert image to HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define initial HSV values for detecting orange color
lower_orange = np.array([5, 150, 150])   # Lower bound for orange in HSV
upper_orange = np.array([15, 255, 255])  # Upper bound for orange in HSV

# Function to update the color filter mask dynamically
def update_color_filter(val):
    global lower_orange, upper_orange

    # Get the HSV values from the trackbars
    h_min = cv2.getTrackbarPos("H Min", "Masked Image (Orange Areas)")
    s_min = cv2.getTrackbarPos("S Min", "Masked Image (Orange Areas)")
    v_min = cv2.getTrackbarPos("V Min", "Masked Image (Orange Areas)")
    
    h_max = cv2.getTrackbarPos("H Max", "Masked Image (Orange Areas)")
    s_max = cv2.getTrackbarPos("S Max", "Masked Image (Orange Areas)")
    v_max = cv2.getTrackbarPos("V Max", "Masked Image (Orange Areas)")
    
    # Update lower and upper bounds based on trackbar positions
    lower_orange = np.array([h_min, s_min, v_min])
    upper_orange = np.array([h_max, s_max, v_max])

    # Create a mask with the updated HSV range
    mask = cv2.inRange(hsv, lower_orange, upper_orange)

    # Apply mask to the image
    masked_image = cv2.bitwise_and(image, image, mask=mask)

    # Show the masked image
    cv2.imshow("Masked Image (Orange Areas)", masked_image)

# Create a window for the masked image with trackbars
cv2.namedWindow("Masked Image (Orange Areas)")

# Create trackbars for adjusting the HSV values dynamically
cv2.createTrackbar("H Min", "Masked Image (Orange Areas)", 1, 180, update_color_filter)
cv2.createTrackbar("S Min", "Masked Image (Orange Areas)", 150, 255, update_color_filter)
cv2.createTrackbar("V Min", "Masked Image (Orange Areas)", 150, 255, update_color_filter)

cv2.createTrackbar("H Max", "Masked Image (Orange Areas)", 15, 180, update_color_filter)
cv2.createTrackbar("S Max", "Masked Image (Orange Areas)", 255, 255, update_color_filter)
cv2.createTrackbar("V Max", "Masked Image (Orange Areas)", 255, 255, update_color_filter)

# Initialize the color filter with default values
update_color_filter(0)

# Convert the image to grayscale and detect edges for Hough transform
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150)
dilated_edges = cv2.dilate(edges, None, iterations=1)  # Dilate the edges for better line detection

# Create a window for the final result with trackbars for Hough Transform parameters
cv2.namedWindow("Detected Grid and Corners")

# Set default values for the trackbars for Hough Transform
def update_hough_line_transform(val):
    threshold = cv2.getTrackbarPos("Threshold", "Detected Grid and Corners")
    min_line_length = cv2.getTrackbarPos("Min Line Length", "Detected Grid and Corners")
    max_line_gap = cv2.getTrackbarPos("Max Line Gap", "Detected Grid and Corners")

    # Use Hough Line Transform to detect lines
    lines = cv2.HoughLinesP(dilated_edges, 1, np.pi / 180, threshold=threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

    # Store detected lines and intersection points
    detected_lines = []
    intersection_points = []

    if lines is not None:
        # Clear the previous lines and intersections
        image_copy = image.copy()

        for line in lines:
            x1, y1, x2, y2 = line[0]
            detected_lines.append(((x1, y1), (x2, y2)))
            cv2.line(image_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Find intersections of the lines (possible corners)
        for i in range(len(detected_lines)):
            for j in range(i + 1, len(detected_lines)):
                x1, y1 = detected_lines[i][0]
                x2, y2 = detected_lines[i][1]
                x3, y3 = detected_lines[j][0]
                x4, y4 = detected_lines[j][1]

                # Calculate intersection point
                denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
                if denominator != 0:
                    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
                    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator
                    intersection_points.append((int(intersect_x), int(intersect_y)))
                    cv2.circle(image_copy, (int(intersect_x), int(intersect_y)), 4, (0, 0, 255), -1)

        # Convert to NumPy array for clustering
        points = np.array(intersection_points)

        # Filter valid points within image bounds
        valid_points = points[(points[:, 0] > 0) & (points[:, 0] < width) & (points[:, 1] > 0) & (points[:, 1] < height)]

        # Identify the four corners using KMeans clustering
        if len(valid_points) > 3:
            kmeans = KMeans(n_clusters=4, n_init=10)
            kmeans.fit(valid_points)
            cluster_centers = kmeans.cluster_centers_

            # Sort corners based on position
            sorted_corners = sorted(cluster_centers, key=lambda c: (c[1], c[0]))  # Sort by Y, then X

            top_two = sorted(sorted_corners[:2], key=lambda c: c[0])  # Sort top corners by X
            bottom_two = sorted(sorted_corners[2:], key=lambda c: c[0])  # Sort bottom corners by X

            # Assign corner names
            corners = {
                "Top Left": top_two[0],
                "Top Right": top_two[1],
                "Bottom Left": bottom_two[0],
                "Bottom Right": bottom_two[1],
            }

            # Print corner positions
            print("\nDetected Corners:")
            for name, (x, y) in corners.items():
                print(f"{name}: ({int(x)}, {int(y)})")
                cv2.circle(image_copy, (int(x), int(y)), 7, (255, 0, 0), -1)  # Blue color
                cv2.putText(image_copy, name, (int(x) - 40, int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            # Find bounding box of the corners
            min_x = min(c[0] for c in cluster_centers)
            max_x = max(c[0] for c in cluster_centers)
            min_y = min(c[1] for c in cluster_centers)
            max_y = max(c[1] for c in cluster_centers)

            # Grid dimensions
            num_cells = 20
            cell_width = (max_x - min_x) / num_cells
            cell_height = (max_y - min_y) / num_cells

            # Draw grid within bounding box
            for i in range(1, num_cells):
                x = min_x + i * cell_width
                cv2.line(image_copy, (int(x), int(min_y)), (int(x), int(max_y)), (255, 255, 255), 1)

                y = min_y + i * cell_height
                cv2.line(image_copy, (int(min_x), int(y)), (int(max_x), int(y)), (255, 255, 255), 1)

            # Draw a black outline around the grid
            cv2.rectangle(image_copy, (int(min_x), int(min_y)), (int(max_x), int(max_y)), (0, 0, 0), 2)

        # Show the results with updated parameters
        cv2.imshow("Detected Grid and Corners", image_copy)

# Create trackbars for adjusting the Hough Transform parameters
cv2.createTrackbar("Threshold", "Detected Grid and Corners", 144, 255, update_hough_line_transform)
cv2.createTrackbar("Min Line Length", "Detected Grid and Corners", 500, 500, update_hough_line_transform)
cv2.createTrackbar("Max Line Gap", "Detected Grid and Corners", 10, 50, update_hough_line_transform)

# Initialize the display with the default values of the trackbars
update_hough_line_transform(0)

# Wait until the user closes the window
cv2.waitKey(0)
cv2.destroyAllWindows()


# This script:
**- Detects walls**

**- Creates coordinate system**

**- Detects balls within coordinate system**

**- Puts detected balls into seperatie lists.('c' to print)**




In [ ]:
'''
This program processes an image to detect a field, its boundaries, and balls (both orange and white) on the field.

Steps:
1. The image is resized for consistency and converted to HSV color space to allow better color-based filtering.
2. The program isolates the orange field boundaries by using color thresholds in the HSV color space.
3. Edges are detected using Canny edge detection, followed by Hough Line Transform to identify lines representing walls on the field.
4. Intersections of the detected lines are calculated to find the corners of the field.
5. KMeans clustering is applied to group the intersection points into four corners (Top-left, Top-right, Bottom-left, Bottom-right).
6. A grid is drawn within the field's bounding box, with adjustable grid size.
7. The program detects orange and white balls on the field, maps them to grid coordinates, and displays them on the image.
8. The user can interact with the program by pressing:
   - 'c' to print the detected ball positions to the console.
   - 'q' to exit the program.

The final image will display the detected field, corners, grid, and balls with their respective grid positions.
'''


import cv2
import numpy as np
from sklearn.cluster import KMeans

# Load image
image = cv2.imread("Udklip.PNG")
height, width = image.shape[:2]

# Resize for consistency
new_width = 800
new_height = int((new_width / width) * height)
image = cv2.resize(image, (new_width, new_height))

# Convert to HSV for better color filtering
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Detect orange field boundaries
lower_orange = np.array([5, 150, 150])
upper_orange = np.array([15, 255, 255])
mask = cv2.inRange(hsv, lower_orange, upper_orange)

# Edge detection and line detection for walls
edges = cv2.Canny(mask, 50, 150)
dilated_edges = cv2.dilate(edges, None, iterations=1)
lines = cv2.HoughLinesP(dilated_edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=10)

# Find intersections (field corners)
intersection_points = []
detected_lines = []
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        detected_lines.append(((x1, y1), (x2, y2)))

    # Compute intersections
    for i in range(len(detected_lines)):
        for j in range(i + 1, len(detected_lines)):
            x1, y1 = detected_lines[i][0]
            x2, y2 = detected_lines[i][1]
            x3, y3 = detected_lines[j][0]
            x4, y4 = detected_lines[j][1]

            # Compute intersection
            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
            if denominator != 0:
                intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
                intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator
                intersection_points.append((int(intersect_x), int(intersect_y)))

# Cluster and sort field corners
points = np.array(intersection_points)
valid_points = points[(points[:, 0] > 0) & (points[:, 0] < width) & (points[:, 1] > 0) & (points[:, 1] < height)]
if len(valid_points) > 3:
    kmeans = KMeans(n_clusters=4, n_init=10)
    kmeans.fit(valid_points)
    cluster_centers = kmeans.cluster_centers_

    # Sort corners: Top-left, Top-right, Bottom-left, Bottom-right
    sorted_corners = sorted(cluster_centers, key=lambda c: (c[1], c[0]))
    top_two = sorted(sorted_corners[:2], key=lambda c: c[0])
    bottom_two = sorted(sorted_corners[2:], key=lambda c: c[0])

    corners = {
        "Top Left": top_two[0],
        "Top Right": top_two[1],
        "Bottom Left": bottom_two[0],
        "Bottom Right": bottom_two[1],
    }

    # Define field boundaries
    min_x = min(c[0] for c in cluster_centers)
    max_x = max(c[0] for c in cluster_centers)
    min_y = min(c[1] for c in cluster_centers)
    max_y = max(c[1] for c in cluster_centers)

    # Grid Settings
    num_cells = 100
    cell_width = (max_x - min_x) / num_cells
    cell_height = (max_y - min_y) / num_cells

    # Draw 20x20 grid
    for i in range(1, num_cells):
        x = min_x + i * cell_width
        y = min_y + i * cell_height
        cv2.line(image, (int(x), int(min_y)), (int(x), int(max_y)), (255, 255, 255), 1)
        cv2.line(image, (int(min_x), int(y)), (int(max_x), int(y)), (255, 255, 255), 1)

# Ball detection setup
white_balls_detected = []  # List to store white balls' positions
orange_balls_detected = []  # List to store orange balls' positions

def detect_ball(image, hsv, detect_orange):
    """Detects the ball and maps it to grid coordinates."""
    ball_lower = np.array([15, 190, 190]) if detect_orange else np.array([0, 0, 180])
    ball_upper = np.array([25, 255, 255]) if detect_orange else np.array([180, 50, 255])
    
    mask_ball = cv2.inRange(hsv, ball_lower, ball_upper)
    blurred = cv2.GaussianBlur(mask_ball, (9, 9), 2)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=15, minRadius=5, maxRadius=20)
    
    detected_balls = []
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for (x, y, r) in circles[0, :]:
            cv2.circle(image, (x, y), r, (0, 255, 0), 2)
            
            # Map ball to grid
            grid_col = int(((x - min_x) / (max_x - min_x)) * num_cells)
            grid_row = int(((y - min_y) / (max_y - min_y)) * num_cells)
            
            # Store the detected ball based on its color
            ball_position = (grid_col, grid_row)
            if detect_orange:
                cv2.putText(image, f"o({grid_col}, {grid_row})", (x + 10, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                if ball_position not in orange_balls_detected:
                    orange_balls_detected.append(ball_position)
            else:
                cv2.putText(image, f"w({grid_col}, {grid_row})", (x + 10, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                if ball_position not in white_balls_detected:
                    white_balls_detected.append(ball_position)
                    
            
    return image

# Main loop
while True:
    img_copy = image.copy()
    
    # Detect both white and orange balls
    img_copy = detect_ball(img_copy, hsv, detect_orange=True)  # Detect orange balls
    img_copy = detect_ball(img_copy, hsv, detect_orange=False)  # Detect white balls
    
    # Display detected balls on screen
    cv2.putText(img_copy, f"Orange Balls: {len(orange_balls_detected)}", (30, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.putText(img_copy, f"White Balls: {len(white_balls_detected)}", (30, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("Detected Field & Balls", img_copy)


    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):  # c to print balls detected
        print("Orange Balls:", orange_balls_detected)
        print("White Balls:", white_balls_detected)

    elif key == ord('q'):  # q to exit
        break

cv2.destroyAllWindows()


# This script uses video to detect balls

In [ ]:
''' 
This code captures live video from camera, processes the frames to detect either white or orange balls 
based on user input, and displays the results with real-time tracking. 

It uses OpenCV to create trackbars for tuning HSV values, adjust HoughCircle parameters, 
and switch between white and orange ball detection. 

The program shows three windows: one for ball detection, one for a binary mask of the color filter, 
and one for controlling parameters. 

Pressing 'b' switches between white and orange ball detection, while 'q' stops the capture and closes the windows.
'''


import cv2
import numpy as np

# Boolean to switch between white and orange ball detection
white = False  

# Create windows
cv2.namedWindow("Trackbars", cv2.WINDOW_NORMAL)
cv2.namedWindow("Ball Detection", cv2.WINDOW_NORMAL)
cv2.namedWindow("Binary Mask", cv2.WINDOW_NORMAL)

# Move windows
cv2.moveWindow("Trackbars", 0, 0)
cv2.moveWindow("Ball Detection", 605, 0)
cv2.moveWindow("Binary Mask", 1100, 0)

# Function to do nothing (for trackbars)
def nothing(x):
    pass

# Function to create trackbars based on ball color
def create_trackbars(white):
    if white:
        cv2.createTrackbar("Hue min", "Trackbars", 0, 180, nothing)
        cv2.createTrackbar("Hue max", "Trackbars", 180, 180, nothing)
        cv2.createTrackbar("Sat min", "Trackbars", 0, 255, nothing)
        cv2.createTrackbar("Sat max", "Trackbars", 20, 255, nothing)
        cv2.createTrackbar("Bright min", "Trackbars", 180, 255, nothing)
        cv2.createTrackbar("Bright max", "Trackbars", 255, 255, nothing)
    else:
        cv2.createTrackbar("Hue min", "Trackbars", 15, 180, nothing)
        cv2.createTrackbar("Hue max", "Trackbars", 25, 180, nothing)
        cv2.createTrackbar("Sat min", "Trackbars", 190, 255, nothing)
        cv2.createTrackbar("Sat max", "Trackbars", 255, 255, nothing)
        cv2.createTrackbar("Bright min", "Trackbars", 190, 255, nothing)
        cv2.createTrackbar("Bright max", "Trackbars", 255, 255, nothing)

# Initialize trackbars for white balls
create_trackbars(white)

# Trackbars for HoughCircles parameters
cv2.createTrackbar("p1 Canny Edge", "Trackbars", 10, 255, nothing)
cv2.createTrackbar("p2 Sensitivity", "Trackbars", 10, 100, nothing)
cv2.createTrackbar("Ball min", "Trackbars", 4, 100, nothing)
cv2.createTrackbar("Ball max", "Trackbars", 10, 200, nothing)

# Capture from camera and show images with detection
def capture_from_camera_and_show_images():

    global white
    
    print("Starting image capture")

    print("Opening connection to camera")
    cap = cv2.VideoCapture(0)  # Use 0 for default camera
    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    print("Starting camera loop")
    stop = False
    while not stop:
        ret, new_frame = cap.read()
        if not ret:
            print("Can't receive frame. Exiting ...")
            break

        # Convert frame to HSV
        hsv = cv2.cvtColor(new_frame, cv2.COLOR_BGR2HSV)

        # Get trackbar values
        lower_h = cv2.getTrackbarPos("Hue min", "Trackbars")
        upper_h = cv2.getTrackbarPos("Hue max", "Trackbars")
        lower_s = cv2.getTrackbarPos("Sat min", "Trackbars")
        upper_s = cv2.getTrackbarPos("Sat max", "Trackbars")
        lower_v = cv2.getTrackbarPos("Bright min", "Trackbars")
        upper_v = cv2.getTrackbarPos("Bright max", "Trackbars")

        param1 = max(1, cv2.getTrackbarPos("p1 Canny Edge", "Trackbars"))
        param2 = max(1, cv2.getTrackbarPos("p2 Sensitivity", "Trackbars"))
        min_radius = cv2.getTrackbarPos("Ball min", "Trackbars")
        max_radius = cv2.getTrackbarPos("Ball max", "Trackbars")

        # Apply color filter
        lower_bound = np.array([lower_h, lower_s, lower_v])
        upper_bound = np.array([upper_h, upper_s, upper_v])
        mask = cv2.inRange(hsv, lower_bound, upper_bound)
        blurred = cv2.GaussianBlur(mask, (9, 9), 2)

        # Detect circles
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, 1, 20,
                                   param1=param1, param2=param2, 
                                   minRadius=min_radius, maxRadius=max_radius)

        # Draw detected balls on the original image
        output = new_frame.copy()
        ball_count = 0
        positions_text = "No balls detected"  # Default value for positions_text

        if circles is not None:
            circles = np.uint16(np.around(circles))
            ball_count = len(circles[0, :])

            # Prepare position info
            positions_text = [f"({x}, {y})" for x, y, r in circles[0, :]]  # Adjust to global coordinates
            positions_text = ", ".join(positions_text)

            for (x, y, r) in circles[0, :] :
                cv2.circle(output, (x, y), r, (0, 255, 0), 2)
                cv2.putText(output, "Ball", (x, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Display ball count and positions on screen
        cv2.putText(output, f"Balls detected: {ball_count}", (10, new_frame.shape[0] - 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(output, "Positions: " + positions_text, (10, new_frame.shape[0] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        # Show the original frame with detected circles
        cv2.imshow("Ball Detection", output)
        cv2.imshow("Binary Mask", mask)

        # Switch between detecting white and orange balls when 'b' is pressed
        key = cv2.waitKey(1) & 0xFF
        if key == ord('b'):
            white = not white
            create_trackbars(white)

        # Press 'q' to exit
        if key == ord('q'):
            stop = True

    print("Stopping image loop")
    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    capture_from_camera_and_show_images()
